In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd
import torch
from torch.utils.data import Dataset

train_df = pd.read_csv("train_2kmZucJ.csv")
test_df = pd.read_csv("test_oJQbWVk.csv")

2025-05-23 17:08:45.427216: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748000325.880682   13219 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748000325.993500   13219 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748000326.918782   13219 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748000326.918859   13219 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748000326.918862   13219 computation_placer.cc:177] computation placer alr

In [4]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df["tweet"].tolist(), train_df["label"].tolist(), test_size=0.1, stratify=train_df["label"], random_state=42
)

model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_texts, train_labels, tokenizer)
val_dataset = TweetDataset(val_texts, val_labels, tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=False, 
    logging_dir="./logs",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,

)

trainer.train()


/home/tuaalab/NLP/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
500,0.274500
1000,0.171400


/home/tuaalab/NLP/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=1338, training_loss=0.20020328116880226, metrics={'train_runtime': 9821.5855, 'train_samples_per_second': 2.177, 'train_steps_per_second': 0.136, 'total_flos': 1406604331173888.0, 'train_loss': 0.20020328116880226, 'epoch': 3.0})

In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_path = "/home/tuaalab/Sentiment Analysis/results/checkpoint-1338"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment") 

In [7]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from tqdm import tqdm  

# Tokenize without tensor return (so we can batch later)
test_encodings = tokenizer(test_df["tweet"].tolist(), truncation=True, padding=True, return_tensors="pt")

# Move model to device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Prepare DataLoader for batching
dataset = TensorDataset(
    test_encodings["input_ids"],
    test_encodings["attention_mask"]
)
loader = DataLoader(dataset, batch_size=32)  

# Run inference in batches
all_preds = []
with torch.no_grad():
    for batch in tqdm(loader):
        input_ids, attention_mask = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, axis=1)
        all_preds.extend(preds.cpu().numpy())

# Save predictions
submission = pd.DataFrame({"id": test_df["id"], "label": all_preds})
submission.to_csv("Predictions_Final.csv", index=False)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
100%|██████████| 62/62 [03:25<00:00,  3.31s/it]
